In [1]:
import torch
import torch
import torch.utils.data
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import TQDMProgressBar

from livecellx.model_zoo.segmentation.sc_correction import CorrectSegNet
import numpy as np
import warnings
import tqdm
torch.manual_seed(237)


In [2]:
# torch.cuda.is_available(), torch.cuda.current_device(), torch.cuda.device_count()

import torch
import torch
import torch.utils.data
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import TQDMProgressBar

from livecellx.model_zoo.segmentation.sc_correction import CorrectSegNet
from livecellx.model_zoo.segmentation.sc_correction_dataset import CorrectSegNetDataset
torch.manual_seed(237)

In [3]:
from pathlib import Path
import pandas as pd
# train_dir = Path("./notebook_results/a549_ccp_vim/train_data_v4/")
# test_dir = Path("./notebook_results/a549_ccp_vim/test_data_v4/")
# train_dir = Path("./notebook_results/a549_ccp_vim/train_data_v5/")
# test_dir = Path("./notebook_results/a549_ccp_vim/test_data_v5/")

# train_dir = Path("./notebook_results/a549_ccp_vim/train_data_v8/")
# test_dir = Path("./notebook_results/a549_ccp_vim/test_data_v8/")
# train_csv = train_dir / "train_data.csv"

# train_csv = "./notebook_results/a549_ccp_vim/train_real_td_data/real_underseg_td1_XY8/data.csv"
# train_csv = "./notebook_results/a549_ccp_vim/train_real_td_data/real_overseg_td1_XY9/data.csv"
# train_csv = "./notebook_results/a549_ccp_vim/train_data_v11/test_overseg_dropout/data.csv"
# train_csv = "/home/ken67/LiveCellTracker-dev/notebooks/notebook_results/a549_ccp_vim/test_data_v11/real_overseg_td1_XY5_dropout/data.csv"
train_csv = "/home/ken67/LiveCellTracker-dev/notebooks/notebook_results/a549_ccp_vim/train_data_v11/train_data.csv"
# train_csv = "/home/ken67/LiveCellTracker-dev/notebooks/notebook_results/a549_ccp_vim/test_data_v11/train_data.csv"
# train_csv = "D:\\LiveCellTracker-dev\\datasets\\yaxuan_csn_annotation\\train_data.csv"
# train_csv = "/home/ken67/LiveCellTracker-dev/notebooks/notebook_results/a549_ccp_vim/train_data_v6/synthetic_overseg/data.csv"
# train_csv = "/home/ken67/LiveCellTracker-dev/notebooks/notebook_results/a549_ccp_vim/train_data_v9/synthetic_overseg/data.csv"
train_df = pd.read_csv(train_csv)

if "subdir" not in train_df.columns:
    train_df["subdir"] = "cur"
# test_df = pd.read_csv(test_dir / "train_data.csv")
train_df[:2]

,raw,seg,gt,raw_seg,scale,aug_diff_mask,gt_label_mask,raw_transformed_img,subdir,Unnamed: 0
0,notebook_results/a549_ccp_vim/train_data_v11/r...,notebook_results/a549_ccp_vim/train_data_v11/r...,notebook_results/a549_ccp_vim/train_data_v11/r...,notebook_results/a549_ccp_vim/train_data_v11/r...,0.0,notebook_results/a549_ccp_vim/train_data_v11/r...,notebook_results/a549_ccp_vim/train_data_v11/r...,notebook_results/a549_ccp_vim/train_data_v11/r...,real_overseg_td1_XY9_dropout,NaN
1,notebook_results/a549_ccp_vim/train_data_v11/r...,notebook_results/a549_ccp_vim/train_data_v11/r...,notebook_results/a549_ccp_vim/train_data_v11/r...,notebook_results/a549_ccp_vim/train_data_v11/r...,0.0,notebook_results/a549_ccp_vim/train_data_v11/r...,notebook_results/a549_ccp_vim/train_data_v11/r...,notebook_results/a549_ccp_vim/train_data_v11/r...,real_overseg_td1_XY9_dropout,NaN


Check all gt labels: any empty?

In [4]:
from livecellx.core.datasets import read_img_default
# check all gt label's unique labels
gt_paths = train_df["gt"].values

for path in gt_paths:
    # read tiff
    gt_tif = read_img_default(path)
    # print("gt shape:", gt_tif.shape)
    # print("unique labels:", np.unique(gt_tif))
    assert np.unique(gt_tif.flatten()).shape[0] > 1, "#unique labels should be > 1"

In [5]:
from livecellx.model_zoo.segmentation.eval_csn import compute_metrics, assemble_train_test_dataset

model = CorrectSegNet.load_from_checkpoint(r"/home/ken67/LiveCellTracker-dev/notebooks/lightning_logs/version_v11_02/checkpoints/epoch=3819-global_step=0.ckpt")
train_dataset_eval, val_dataset_eval, test_dataset_eval, whole_dataset_eval = assemble_train_test_dataset(train_df, train_df, model)

/home/ken67/anaconda3/envs/livecell/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ken67/anaconda3/envs/livecell/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


>>> Using MSE loss
>>> Based on loss type, training output threshold:  1
input type: raw_aug_duplicate
if apply_gt_seg_edt: True
whether to normalize_uint8: False
input type: raw_aug_duplicate
if apply_gt_seg_edt: True
whether to normalize_uint8: False


In [6]:
from livecellx.model_zoo.segmentation.eval_csn import assemble_dataset, assemble_train_test_dataset

# split_seed = 237
# dataset = assemble_dataset(train_df, apply_gt_seg_edt = model.apply_gt_seg_edt, exclude_raw_input_bg=model.exclude_raw_input_bg, input_type=model.input_type)
# train_sample_num = int(len(dataset) * 0.8)
# val_sample_num = len(dataset) - train_sample_num
# split_generator = torch.Generator().manual_seed(split_seed)
# train_dataset, val_dataset = torch.utils.data.random_split(
#     dataset, [train_sample_num, val_sample_num], generator=split_generator
# )

# test_dataset = assemble_dataset(test_df, apply_gt_seg_edt = model.apply_gt_seg_edt, exclude_raw_input_bg=model.exclude_raw_input_bg, input_type=model.input_type)
train_dataset = assemble_dataset(train_df, apply_gt_seg_edt = True, exclude_raw_input_bg=False, input_type="raw_aug_duplicate")

input type: raw_aug_duplicate
if apply_gt_seg_edt: True
whether to normalize_uint8: False


In [16]:
from torchvision import transforms
train_dataset.transform =     transforms.Compose(
        [
            transforms.Resize((412, 412))
        ]
    )

Viz training dataset

In [7]:
train_dataset.normalize_uint8=False
train_dataset.apply_gt_seg_edt = True

In [8]:
len(train_dataset), len(train_dataset.raw_img_paths)

(1455, 1455)

In [9]:
for idx in range(len(train_dataset)):
    sample = train_dataset[idx]
    gt_mask = sample["gt_mask"]
    assert np.unique(gt_tif.flatten()).shape[0] > 1, "#unique labels should be > 1"

In [10]:
train_dataset[0]["input"].shape

torch.Size([3, 131, 103])

In [21]:
from livecellx.model_zoo.segmentation.eval_csn import compute_metrics
import importlib
import livecellx.model_zoo.segmentation.eval_csn
importlib.reload(livecellx.model_zoo.segmentation.eval_csn)

model.cuda()
model.eval()
metrics = livecellx.model_zoo.segmentation.eval_csn.compute_metrics(train_dataset, model, whole_dataset=train_dataset)

  0%|          | 0/1455 [00:00<?, ?it/s]

/home/ken67/anaconda3/envs/livecell/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
  0%|          | 0/1455 [00:00<?, ?it/s]


sample input shape: torch.Size([3, 412, 412])
model input shape: torch.Size([1, 3, 412, 412])


AssertionError: 

In [ ]:
import matplotlib.pyplot as plt
import random

def viz_sample_only(sample):
    fig, axes = plt.subplots(1, 7, figsize=(3 * 7, 3))
    ax_idx = 0
    ax = axes[ax_idx]
    ax.imshow(sample["input"][0])
    ax.set_title("input: dim0")

    ax_idx += 1
    ax = axes[ax_idx]
    ax.imshow(sample["input"][1])
    ax.set_title("input: dim1")

    ax_idx += 1
    ax = axes[ax_idx]
    ax.imshow(sample["input"][2])
    ax.set_title("input:dim2")

    # gt
    ax_idx += 1
    ax = axes[ax_idx]
    ax.imshow(sample["gt_mask"][0])
    ax.set_title("gt0 seg")

    ax_idx += 1
    ax = axes[ax_idx]
    ax.imshow(sample["gt_mask"][1])
    ax.set_title("gt1 seg")

    ax_idx += 1
    ax = axes[ax_idx]
    ax.imshow(sample["gt_mask"][2])
    ax.set_title("gt2 seg")

    ax_idx += 1
    ax = axes[ax_idx]
    ax.imshow(sample["gt_label_mask"])
    ax.set_title("gt label mask")

    for ax in axes:
        ax.axis("off")

out_dir = Path("./tmp/csn_check_data_v11/")
out_dir.mkdir(exist_ok=True, parents=True)
for i in range(50):
    sample_id = random.randint(0, len(train_dataset) - 1)
    sample = train_dataset[sample_id]
    df_row = train_dataset.raw_df.iloc[sample_id]
    print("source subdir:", df_row["subdir"])
    print("sample id:", sample_id)
    # plt.hist(sample["input"][0].flatten(), bins=100)
    # plt.hist(sample["gt_mask"][1].flatten(), bins=100)
    viz_sample_only(sample)
    plt.suptitle(f"sample id: {sample_id}, raw_df subdir: {df_row['subdir']}, raw_path: {df_row['raw']}")
    # plt.subplots_adjust(top=0.55)
    plt.tight_layout()
    plt.axis("off")
    # plt.show()
    plt.savefig(out_dir / f"sample_{sample_id}.png")
    
    

In [ ]:
for sample in tqdm.tqdm(train_dataset):
    gt_label_mask = sample["gt_label_mask"]
    assert np.unique(gt_label_mask).shape[0] > 0